# Libraries


In [0]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [0]:
#sc=SparkContext() #Uncomment this line if you're creating a SparkContext() for the first time
ss=SparkSession(sc)

# Rename

In [0]:
dbutils.fs.cp('/FileStore/tables/large_csv.gz', '/FileStore/tables/large.csv.gz')

Out[3]: True

# Question 1

In [0]:
large=ss.read.csv('/FileStore/tables/large.csv.gz', header=True)

In [0]:
large.registerTempTable('large')

/databricks/spark/python/pyspark/sql/dataframe.py:331: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


## Answer

In [0]:
ss.sql('select count(distinct sentence) large from large').show()

+------+
| large|
+------+
|389639|
+------+



# Question 2

In [0]:
large_sentence=ss.sql('select sentence from large')

In [0]:
large_sentence.repartition(1).write.csv('/FileStore/tables/large_sentence', mode='overwrite')

In [0]:
large_sentence=sc.textFile('/FileStore/tables/large_sentence')

## Answer

In [0]:
large_sentence.map(lambda x: (len(x.split(' ')), x)).sortBy(lambda x: x[0], ascending=False).map(lambda x: x[0]).take(10)

Out[10]: [4571, 2499, 562, 528, 426, 413, 382, 381, 348, 335]

# Question 3

In [0]:
average_number_bigrams_large_sentence=large_sentence.map(lambda x: len(x.split(' '))-1).map(lambda x: [x, 1]).reduce(lambda x, y: [x[0]+y[0], x[1]+y[1]])

## Answer

In [0]:
average_number_bigrams_large_sentence[0]/average_number_bigrams_large_sentence[1]

Out[12]: 18.036295

# Question 4

In [0]:
large_sentence=large_sentence.map(lambda x: x.split(' '))

In [0]:
def concatenate_word(sentence):
    my_list=[]
    for i in range(0, len(sentence)-1):
        j=i+1
        my_list.append(sentence[i]+' '+sentence[j])
    return my_list

In [0]:
large_sentence=large_sentence.flatMap(concatenate_word)

## Answer

In [0]:
large_sentence.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[1], ascending=False).take(10)

Out[16]: [('of the', 76290),
 ('in the', 54056),
 ('to the', 25484),
 ('at the', 21588),
 ('is a', 19261),
 ('for the', 17942),
 ('on the', 16049),
 ('and the', 15822),
 ('as a', 13240),
 ('with the', 11928)]

# Question 5

In [0]:
magpie=ss.read.json('/FileStore/tables/MAGPIE_unfiltered.jsonl')

In [0]:
magpie.registerTempTable('magpie')

In [0]:
large_sentence.map(lambda x: [x]).toDF(['bigrams']).registerTempTable('large_bigrams')

/databricks/spark/python/pyspark/sql/dataframe.py:331: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


## Answer


In [0]:
ss.sql('select count(distinct bigrams) large from large_bigrams where bigrams in (select idiom from magpie)').show()

+-----+
|large|
+-----+
|   67|
+-----+



# Question 6


In [0]:
large_sentence.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y).toDF(['bigrams', 'frequency']).registerTempTable('large_bigrams_frequency')

## Answer


In [0]:
ss.sql('select * from (select *, dense_rank() over(order by frequency desc, bigrams asc) rank from large_bigrams_frequency where bigrams not in (select idiom from magpie)) where rank between 2500 and 2510').show()

+----------------+---------+----+
|         bigrams|frequency|rank|
+----------------+---------+----+
|         2018 to|      174|2500|
| Communist Party|      174|2501|
|Cup competition.|      174|2502|
|      During the|      174|2503|
|       a meeting|      174|2504|
|         a state|      174|2505|
|      a two-year|      174|2506|
|           at St|      174|2507|
|  been described|      174|2508|
|       bishop of|      174|2509|
|        chose to|      174|2510|
+----------------+---------+----+

